In [1]:
import os
import time
import glob
import shutil
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, BatchNormalization, Flatten, Conv1D, Permute
from tensorflow.keras.layers import MaxPooling1D, Dropout, Reshape, Multiply, Conv2D, MaxPool2D, LSTM, Add, Lambda, AveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomTranslation, RandomRotation, RandomZoom
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import SGD, Nadam, RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
from tensorflow.keras.applications import ResNet50, ResNet152
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import EfficientNetB7, EfficientNetB4, EfficientNetB2, EfficientNetB1


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
real_path = 'F:/deepfake_1st/real_MTCNN'
fake_path = 'F:/fake_MTCNN_V2' 


file = []
for filename in os.listdir(real_path ):
    file.append(real_path + '/' + filename)
    
real_df = pd.DataFrame(columns = ['file','label'])
real_df['file'] = file
real_df['label'] = 0

In [3]:
file = []
for filename in os.listdir(fake_path ):
    file.append(fake_path + '/' + filename)
    
fake_df = pd.DataFrame(columns = ['file','label'])
fake_df['file'] = file
fake_df['label'] = 1

In [4]:
df = pd.concat([real_df, fake_df], axis=0)
train_df, valid_df = train_test_split(df, test_size=0.35, random_state = 25, stratify = df['label'])

print(train_df['label'].value_counts())
print(valid_df['label'].value_counts())

1    72399
0    41475
Name: label, dtype: int64
1    38985
0    22333
Name: label, dtype: int64


In [5]:
batch_size = 20
image_size = (300,300)

train_datagen = ImageDataGenerator(rescale=1/255,)
validation_datagen = ImageDataGenerator(rescale=1/255,)

train_gen  = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'file',
    y_col = 'label',
    target_size=image_size,    
    batch_size=batch_size,                                    
    class_mode='raw',
    shuffle=True
    )

val_gen  = validation_datagen.flow_from_dataframe(
    valid_df,
    x_col = 'file',
    y_col = 'label',
    target_size=image_size,    
    batch_size=batch_size,                                    
    class_mode='raw',
    shuffle=False
    )

Found 113874 validated image filenames.
Found 61318 validated image filenames.


In [6]:
input_shape_2 = (300, 300, 3)
output_size = 1

print(input_shape_2)
print(output_size)

(300, 300, 3)
1


In [7]:
class RandomRollLayer(tf.keras.layers.Layer):
    """Shift data"""

    def __init__(self, roll_limit=0.1, u=0.5, **kwargs):
        super().__init__(**kwargs)
        self.roll_limit = roll_limit
        self.u = u

    def build(self, input_shape):
        self.non_trainable_weights.append(self.roll_limit)
        self.non_trainable_weights.append(self.u)
        super().build(input_shape)

    def call(self, pixels, training=None):
        if training is None:
            training = K.learning_phase()

        if not training:
            return pixels

        if tf.random.uniform(shape=[]) < self.u:
            roll_limit = self.roll_limit * pixels.shape[1]
            roll_limit = tf.cast(roll_limit, tf.int32)
            roll = tf.random.uniform(shape=[], minval=-roll_limit, maxval=roll_limit, dtype=tf.int32)

            pixels = tf.roll(pixels, shift=roll, axis=1)

        if tf.random.uniform(shape=[]) < self.u:
            roll_limit = self.roll_limit * pixels.shape[2]
            roll_limit = tf.cast(roll_limit, tf.int32)
            roll = tf.random.uniform(shape=[], minval=-roll_limit, maxval=roll_limit, dtype=tf.int32)

            pixels = tf.roll(pixels, shift=roll, axis=2)
            
        return pixels

    def get_config(self):
        config = {
            'roll_limit': self.roll_limit,
            'u': self.u,
        }
        config.update(super().get_config())

        return config

class RandomAddLayer(tf.keras.layers.Layer):
    """Add data"""

    def __init__(self, add_limit=0.1, u=0.5, **kwargs):
        super().__init__(**kwargs)
        self.add_limit = add_limit
        self.u = u

    def build(self, input_shape):
        self.non_trainable_weights.append(self.add_limit)
        self.non_trainable_weights.append(self.u)
        super().build(input_shape)

    def call(self, pixels, training=None):
        if training is None:
            training = K.learning_phase()

        if not training:
            return pixels

        if tf.random.uniform(shape=[]) < self.u:
            add = tf.random.uniform(shape=[], minval=-self.add_limit, maxval=self.add_limit, dtype=tf.float32)
            pixels = pixels + add

        return pixels

    def get_config(self):
        config = {
            'add_limit': self.add_limit,
            'u': self.u,
        }
        config.update(super().get_config())

        return config

class RandomMultipleLayer(tf.keras.layers.Layer):
    """Multiple data"""

    def __init__(self, multiple_limit=0.5, u=0.5, **kwargs):
        super().__init__(**kwargs)
        self.multiple_limit = multiple_limit
        self.u = u

    def build(self, input_shape):
        self.non_trainable_weights.append(self.multiple_limit)
        self.non_trainable_weights.append(self.u)
        super().build(input_shape)

    def call(self, pixels, training=None):
        if training is None:
            training = K.learning_phase()

        if not training:
            return pixels

        if tf.random.uniform(shape=[]) < self.u:
            multiple = tf.random.uniform(shape=[], minval=-self.multiple_limit, maxval=self.multiple_limit, dtype=tf.float32)
            pixels = pixels * (1 + multiple)

        return pixels
    
    def get_config(self):
        config = {
            'multiple_limit': self.multiple_limit,
            'u': self.u,
        }
        config.update(super().get_config())

        return config

In [8]:
def thin_resnet_model(input_shape_2, output_size=1, num_clusters=10):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2
    y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
    y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
    y = RandomRotation(factor=(-0.1, 0.1), fill_mode='constant')(y)

    # y = RandomTranslation(0.1, 0.1)(y)
    # y = RandomRotation(0.1)(y)
    # y = RandomZoom(0.1)(y)

    # CONV 1
    y = Conv2D(64, (7, 7), padding='same')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPool2D((2, 2), strides=(2, 2))(y)

    # CONV 2 - 1
    y1 = Conv2D(48, (1, 1), padding='valid')(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(48, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(96, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)
    
    y2 = Conv2D(96, (1, 1), padding='valid')(y)
    y2 = BatchNormalization()(y2)
    y = Add()([y1, y2])
    y = Activation('relu')(y)

    # CONV 2 - 2
    y1 = Conv2D(48, (1, 1), padding='valid')(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(48, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(96, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)
    
    y = Add()([y1, y])
    y = Activation('relu')(y)


    # CONV 3 - 1
    y1 = Conv2D(96, (1, 1), padding='valid', strides=(2, 2))(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(96, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(128, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    y2 = AveragePooling2D((2, 2), strides=2, padding='same')(y)
    y2 = Conv2D(128, (1, 1), padding='valid')(y2)
    # y2 = Conv2D(128, (1, 1), padding='valid', strides=(2, 2))(y)
    y2 = BatchNormalization()(y2)
    y = Add()([y1, y2])
    y = Activation('relu')(y)
  
    # CONV 3 - 2
    y1 = Conv2D(96, (1, 1), padding='valid')(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(96, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(128, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    y = Add()([y1, y])
    y = Activation('relu')(y)

    # CONV 4 - 1
    y1 = Conv2D(128, (1, 1), padding='valid', strides=(2, 2))(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(128, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(256, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    # y2 = Conv2D(256, (1, 1), padding='valid', strides=(2, 2))(y)
    y2 = AveragePooling2D((2, 2), strides=2, padding='same')(y)
    y2 = Conv2D(256, (1, 1), padding='valid')(y2)
    y2 = BatchNormalization()(y2)
    y = Add()([y1, y2])
    y = Activation('relu')(y)

    # CONV 4 - 2
    y1 = Conv2D(128, (1, 1), padding='valid')(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(128, (3, 3), padding='same')(y1)
    y1 = BatchNormalization(gamma_initializer='zeros')(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(256, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    y = Add()([y1, y])
    y = Activation('relu')(y)

    # CONV 5 - 1
    y1 = Conv2D(256, (1, 1), padding='valid', strides=(2, 2))(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(256, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(512, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    # y2 = Conv2D(512, (1, 1), padding='valid', strides=(2, 2))(y)
    y2 = AveragePooling2D((2, 2), strides=2, padding='same')(y)
    y2 = Conv2D(512, (1, 1), padding='valid')(y2)
    y2 = BatchNormalization()(y2)
    y = Add()([y1, y2])
    y = Activation('relu')(y)

    # CONV 5 - 2
    y1 = Conv2D(256, (1, 1), padding='valid')(y)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(256, (3, 3), padding='same')(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation('relu')(y1)
    y1 = Conv2D(512, (1, 1), padding='valid')(y1)
    y1 = BatchNormalization()(y1)

    y = Add()([y1, y])
    y = Activation('relu')(y)

    # CONV 6
    y = Conv2D(512, (2, 2), padding='valid')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    # y = Reshape((-1, 512))(y)
    # y = Flatten()(y)
    y = GlobalAveragePooling2D()(y)
    # y = NetVLAD(num_clusters=num_clusters)(y)
#     y = tf.keras.layers.Concatenate(axis=1)([y, input_1])
    y = Dropout(0.3)(y)

    y = Dense(output_size)(y)
    y = Activation('sigmoid')(y)
    output = y

    model = Model(inputs=[input_2], outputs=output, name='thin_resnet_model')
    # optimizer = tfa.optimizers.AdamW(learning_rate=0.05, weight_decay=0.0001)
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])
    

    return model

In [9]:
from tensorflow.keras.applications import VGG19


def vggnet_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/255.0, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=56, width=56)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
   

    model = VGG19(include_top=False, input_tensor=y, pooling='max', input_shape=y.shape[1:], weights=None)

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)

    output = y

    model = Model(inputs=[input_2], outputs=output, name='vggnet_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model
    

In [10]:
from tensorflow.keras.applications import ResNet101V2


def resnet_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=56, width=56)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
   

    model = ResNet101V2(include_top=False, input_tensor=y, pooling='max', input_shape=y.shape[1:], weights=None)

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)

    output = y

    model = Model(inputs=[input_2], outputs=output, name='resnet_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model
    

In [11]:
from tensorflow.keras.applications import DenseNet121


def densenet_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=112, width=112)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
#     y = RandomRotation(factor=(-0.1, 0.1), fill_mode='constant')(y)

    model = DenseNet121(include_top=False, input_tensor=y, input_shape=y.shape[1:], weights=None, pooling='max')

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    
    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)
    output = y

    model = Model(inputs=[input_2], outputs=output, name='densenet_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model


In [12]:
from tensorflow.keras.applications import Xception


def xception_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=112, width=112)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
#     y = RandomRotation(factor=(-0.1, 0.1), fill_mode='constant')(y)

    model = Xception(include_top=False, input_tensor=y, input_shape=y.shape[1:], weights=None, pooling='max')

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)
    output = y

    model = Model(inputs=[input_2], outputs=output, name='xception_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model


In [13]:
from tensorflow.keras.applications import InceptionV3


def inception_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=112, width=112)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
#     y = RandomRotation(factor=(-0.1, 0.1), fill_mode='constant')(y)

    model = InceptionV3(include_top=False, input_tensor=y, input_shape=y.shape[1:], weights=None, pooling='max')

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)
    output = y

    model = Model(inputs=[input_2], outputs=output, name='inception_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model

In [14]:
from tensorflow.keras.applications import MobileNetV2


def MobileNetV2_model(input_shape_2, output_size=1):
    input_2 = Input(shape=input_shape_2)  # pixel

    y = input_2

#     y = Rescaling(scale=1.0/127.5, offset=-1.0)(y)
#     y = tf.keras.layers.experimental.preprocessing.Resizing(height=112, width=112)(y)
#     y = RandomRollLayer(roll_limit=0.2, u=0.8)(y)
#     y = RandomRotation(factor=(-0.1, 0.1), fill_mode='constant')(y)

    model = MobileNetV2(include_top=False, input_tensor=y, input_shape=y.shape[1:], weights=None, pooling='max')

    y = model.output
    y = Dense(1024)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(256)(y)
    y = Dropout(0.25)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Dense(output_size, activation='sigmoid', name='softmax')(y)
    output = y

    model = Model(inputs=[input_2], outputs=output, name='inception_model')
    model.compile(loss='mse', optimizer='Nadam', metrics=['accuracy'])

    return model

In [15]:
def conv5_05drop(input_shape_2, output_size=1):
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation="relu", input_shape=(300, 300 , 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(512, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Fully Connected 
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer=Nadam(learning_rate=1e-3),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

In [16]:
def C7L2Nadam(input_shape_2, output_size=1):
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation="relu", input_shape=(300, 300 , 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(512, (3,3), activation="relu"))
    
    model.add(Conv2D(512, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Fully Connected 
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer=Nadam(learning_rate=1e-4),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])   
    return model

In [17]:
def C5L2Nadam_dropout(input_shape_2, output_size=1):
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation="relu", input_shape=(300, 300 , 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(512, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Fully Connected 
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer=Nadam(learning_rate=1e-4),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])  
    return model

In [18]:
def dog_cat_Nadam(input_shape_2, output_size=1):
    model=Sequential()
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=(300,300,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    model.add(Dense(units=512,
                    activation='relu'))
    
    model.add(Dense(units=1,activation='sigmoid'))

    model.compile(optimizer=Nadam(learning_rate=1e-4),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

In [19]:
def dog_cat_3C_Nadam(input_shape_2, output_size=1):
    model=Sequential()
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=(300,300,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    model.add(Dense(units=512,
                    activation='relu'))
    
    model.add(Dense(units=1,activation='sigmoid'))

    model.compile(optimizer=Nadam(learning_rate=1e-4),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

In [20]:
# def k_fold_validation(model_fn, n_splits=5, verbose=1):
#     kf = KFold(n_splits=n_splits)
#     sum_accuracy = 0
#     sum_epoch = 0
#     start_time = time.time()
#     for i, (train_index, val_index) in enumerate(kf.split(train_gen)):
#         model = model_fn(input_shape_2=input_shape_2, output_size=output_size)
#         early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

# #         train_data = [train_letters[train_index], train_pixels[train_index]]
# #         train_label = train_digits[train_index]

# #         val_data = [train_letters[val_index], train_pixels[val_index]]
# #         val_label = train_digits[val_index]

#         history = model.fit(
#             train_gen,
#             epochs=300,
#             validation_data=val_gen,
#             batch_size=4,
#             verbose=verbose,
#             callbacks=[early_stopping],
#         )

#         sum_epoch += len(history.history['val_accuracy'])
#         sum_accuracy += max(history.history['val_accuracy'])
        
#         if verbose >= 0:            
#             print(f'{i+1}/{n_splits} fold result: ')
#             print('epoch num:', len(history.history['val_accuracy']))
#             print('best val accuracy: ', max(history.history['val_accuracy']))
#             print('average 20: ', np.mean(history.history['val_accuracy'][-20:]))
#             print('='*50)

#     print('Average Accuracy: ', sum_accuracy/n_splits)
#     print('Average Epoch: ', sum_epoch/n_splits)
#     print('Time taken: ', time.time() - start_time)
#     print('='*50)

In [21]:
# k_fold_validation(resnet_model, verbose=1)

## 실험 결과

### Augumentation
No Augumentation -> 0.69
RandomRollLayer(roll_limit=0.2, u=0.8)(y) -> 0.835
Roll + ResNet-D -> 0.824
RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode='constant')(y) -> 0.768
RandomRotation(factor=(-0.2, 0.2), fill_mode='constant')(y) -> 0.731
RandomAddLayer(add_limit=0.1, u=0.8)(y) -> 0.751
RandomZoom(height_factor=(0, 0.2), width_factor=(0, 0.2), fill_mode='reflect')(y) -> 0.689

### Model
thin_resnet_model -> 0.835
xception_model -> 0.894
inception_model -> 0.886
densenet_model -> 0.885
vggnet_model -> 0.833
resnet_model -> 0.840

### Loss
RMSProp + categorical_crossentropy -> 0.823

### Batch size
smaller is better

In [22]:
model_fn_list = [
#     conv5_05drop,
#     C7L2Nadam,
#     C5L2Nadam_dropout,
#     dog_cat_Nadam,
    dog_cat_3C_Nadam
#     thin_resnet_model,

#     vggnet_model,
#     MobileNetV2_model,
#     resnet_model,
#     densenet_model,
#     xception_model,
#     inception_model
]

EPOCHS = 300
BATCH_SIZE = 20
CHECKPOINT_PATH = './checkpoint'
MODEL_PATH = './model'

In [23]:
if os.path.isdir(CHECKPOINT_PATH):
    shutil.rmtree(CHECKPOINT_PATH, ignore_errors=True)
os.mkdir(CHECKPOINT_PATH)

if os.path.isdir(MODEL_PATH):
    shutil.rmtree(MODEL_PATH, ignore_errors=True)
os.mkdir(MODEL_PATH)

In [ ]:
for model_fn in model_fn_list:
    kf = KFold(n_splits=5)
    for i, (train_index, val_index) in enumerate(kf.split(train_gen)):
        start_time = time.time()
        model = model_fn(input_shape_2=input_shape_2, output_size=output_size)

        # Validation 점수가 가장 좋은 모델만 저장합니다.
        checkpoint_path = os.path.join(CHECKPOINT_PATH, f'{model.name}_fold{i+1}')
        if os.path.isdir(checkpoint_path):
            shutil.rmtree(checkpoint_path, ignore_errors=True)
        os.mkdir(checkpoint_path)
        checkpoint_file_path = os.path.join(checkpoint_path, 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5')
        checkpoint = ModelCheckpoint(filepath=checkpoint_file_path, monitor='val_accuracy', verbose=0, save_best_only=True)

        # 30회 간 Validation 점수가 좋아지지 않으면 중지합니다.
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

#         train_data = [train_letters[train_index], train_pixels[train_index]]
#         train_label = train_digits[train_index]

#         val_data = [train_letters[val_index], train_pixels[val_index]]
#         val_label = train_digits[val_index]

        history = model.fit(
            train_gen,
            epochs=EPOCHS,
            validation_data=val_gen,
            batch_size=BATCH_SIZE,
            verbose=1,
            callbacks=[early_stopping, checkpoint],
        )

        # 가장 좋은 모델의 weight를 불러옵니다.
        weigth_file = glob.glob('{}/*.hdf5'.format(checkpoint_path))[-1]
        model.load_weights(weigth_file)
        model.save(os.path.join(MODEL_PATH, f'{model.name}_{i+1}.h5'))
        
        shutil.rmtree(checkpoint_path, ignore_errors=True)

        epoch_num = len(history.history['val_accuracy'])
        max_accuracy = max(history.history['val_accuracy'])
        print('='*50)
        print(f'Result of {model.name}, fold {i+1}')
        print(f'Epoch: {epoch_num}')
        print(f'Accuracy: {max_accuracy}')
        print('Time taken: ', time.time() - start_time)
        print('='*50)



In [28]:
model = tf.keras.models.load_model('C:/Users/lhj96/OneDrive/notebook_dir/test_ensemble/model/sequential_4100.h5')

In [16]:
# generator로 predict 돌리기 (새로운 방식)
test_path = 'F:/deepfake_1st/leaderboard_MTCNN'
test_data = [f for f in os.listdir(test_path) if f.endswith('.jpg')]
test_data = sorted(test_data, key=lambda x: int(x[:-4]))
test_datagen = ImageDataGenerator(rescale=1/255,)
test_gen  = test_datagen.flow_from_dataframe(
    pd.DataFrame({'file':test_data}),
    test_path,
    x_col='file',
    y_col='file',
    class_mode = 'raw',
    target_size=(300,300),    
    batch_size=len(test_data)                                      
    )

# paths = []
# for x, y in test_gen:
#     paths = y
#     classes = model.predict(x)
#     break
    
# classes = np.where(classes>0.5,1,0)

# pred_df = pd.DataFrame({'path':list(map(lambda x: int(x[:-4]),paths)),
#                        'y':classes.ravel().astype('int')})
# test = pred_df.sort_values('path')
# # 제출 파일 제작
# sub = pd.read_csv('F:/test/sample_submission.csv')
# sub['y'] = test['y'].values
# sub.to_csv('F:/test/submission.csv', index=False)
# print(sum(classes))

Found 4100 validated image filenames.


In [17]:
import subprocess as sp
import os

def get_gpu_memory():
    total = 7000
    _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]

    ACCEPTABLE_AVAILABLE_MEMORY = 7000
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    print(f"Free: {memory_free_values[0]}MB / {total}MB    {(total - memory_free_values[0]) / total * 100:.2f}% used ")
    
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

get_gpu_memory()

Free: 6999MB / 7000MB    0.01% used 


In [24]:
MODEL_PATH = 'F:/test/model'
model_files = glob.glob(f'{MODEL_PATH}/*.h5')
sum_list = []

for i, model_file in enumerate(model_files):
    pred_list = []
    model = keras.models.load_model(model_file)

    print(i+1, model.name)
    paths = []
    for x, y in test_gen:
        paths = y
        classes = model.predict(x)
        break
    
    classes = np.where(classes>0.6,1,0)

    pred_df = pd.DataFrame({'path':list(map(lambda x: int(x[:-4]),paths)),
                           'y':classes.ravel().astype('int')})
    test = pred_df.sort_values('path')
    # 제출 파일 제작
    sub = pd.read_csv('F:/test/sample_submission.csv')
    sub['y'] = test['y'].values
    sub.to_csv('F:/test/' + str(i) + '_submission.csv', index=False)
    
    model = None
    gc.collect()
#     reset_keras()
    get_gpu_memory()
    print('='*50)
display(sum_list)

1 sequential_11
Free: 2010MB / 7000MB    71.29% used 
2 sequential_12
Free: 2031MB / 7000MB    70.99% used 
3 sequential_14
Free: 2029MB / 7000MB    71.01% used 
4 sequential_15
Free: 2032MB / 7000MB    70.97% used 
5 sequential_16
Free: 2029MB / 7000MB    71.01% used 
6 sequential_1
Free: 2037MB / 7000MB    70.90% used 
7 sequential_2
Free: 2027MB / 7000MB    71.04% used 
8 sequential_3
Free: 2009MB / 7000MB    71.30% used 


[]

In [30]:
sum_list_array=sum_list.copy()
# sum_list_array=sum_list_array[0]
display(sum_list_array)
print(len(sum_list_array[0]))
sum(sum_list_array[0])

[array([0, 1, 0, ..., 0, 0, 4])]

4100


5566

In [31]:
for i in range(len(sum_list_array[0])):
    if sum_list_array[0][i] <= 3:
        sum_list_array[0][i] = 0
    else:
        sum_list_array[0][i] = 1
print(sum(sum_list_array[0]))
# pred_df = pd.DataFrame({'path':list(map(lambda x: int(x[:-4]),paths)),
#                        'y':sum_list_array.ravel().astype('int')})
# test = pred_df.sort_values('path')
# 제출 파일 제작
sub = pd.read_csv('F:/test/sample_submission.csv')
sub['y'] = test['y'].values
sub.to_csv('F:/test/submission.csv', index=False)

933


In [22]:
print(sum(sum_list_array[0]))

933


In [16]:
ensemble_data = np.zeros(pred_list[0].shape)

for pred in pred_list:
    ensemble_data += pred ** 0.5
    
y_pred = np.argmax(ensemble_data, axis=1)

submission_csv = pd.read_csv('F:/test/sample_submission.csv')

submission_csv['y'] = y_pred
submission_csv.to_csv('F:/test/test.csv', index=False)
submission_csv

,path,y
0,leaderboard/image_00000.jpg,0
1,leaderboard/image_00001.jpg,0
2,leaderboard/image_00002.jpg,0
3,leaderboard/image_00003.jpg,0
4,leaderboard/image_00004.jpg,0
...,...,...
4095,leaderboard/image_04095.jpg,0
4096,leaderboard/image_04096.jpg,0
4097,leaderboard/image_04097.jpg,0
4098,leaderboard/image_04098.jpg,0


In [ ]:
df = pd.DataFrame(ensemble_data)
df['max'] = df.iloc[:, 0:10].max(axis=1)
df['pred'] = df.iloc[:, 0:10].idxmax(axis=1)

good_df = df[df['max'] > 20]

test_digits_int = good_df['pred'].to_numpy()
test_digits = tf.keras.utils.to_categorical(test_digits_int)

new_digits = np.concatenate((train_digits, test_digits))
new_letters = np.concatenate((train_letters, test_letters[good_df.index]))
new_pixels = np.concatenate((train_pixels, test_pixels[good_df.index]))
new_digits.shape, new_letters.shape, new_pixels.shape

In [ ]:
train_digits = new_digits
train_letters = new_letters
train_pixels = new_pixels

In [ ]:
test_digits

In [ ]:
y_pred = np.argmax(model.predict([test_letters, test_pixels]), axis=1)
y_pred.shape

In [ ]:
submission_csv['digit'] = y_pred
submission_csv.to_csv('CNN.csv', index=False)
submission_csv
